In [4]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib


import Clone_Model
import rsc

In [5]:
tf.compat.v1.disable_eager_execution()

In [6]:
checkpoint_path = "./cloning_eager_dis/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning_eager_dis/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [14]:
#      CLONE MODEL EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)

K.clear_session() 

tf.compat.v1.disable_eager_execution()

batch_size = None #doesnt work/conflict tensorboard_callback
train, test = rsc.get_titanic_dataset()



model = rsc.get_model(None)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


tensorboard_callback.set_model(model)

history = model.fit(train,
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[])


clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

sat_cb = Clone_Model.sat_results()


history = clone.fit(train,
                    batch_size=batch_size,
                    epochs=1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[sat_cb])
                

Train on 3 steps, validate on 1 steps
3/3 [==============================] - 0s 23ms/step - loss: 3.1866 - val_loss: 1.7910
RESET Weights!
RESET Weights!
RESET Weights!
RESET Weights!
RESET Weights!


ValueError: Error when checking input: expected model_input to have 2 dimensions, but got array with shape (1, 28, 28)

In [37]:
#      CLASSIFY MNIST EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)

K.clear_session() 

tf.compat.v1.disable_eager_execution()

batch_size = 20 

train, test = rsc.get_mnist()

model = rsc.get_model_unitlist(hidden_layers_spec=[128])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
    run_eagerly = tf.executing_eagerly()
    )


tensorboard_callback.set_model(model)

history = model.fit(train[0],
                    train[1],
                    batch_size=batch_size,
                    epochs=5, 
                    steps_per_epoch=None, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[])


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 57us/sample - loss: 2.0756 - accuracy: 0.8500 - val_loss: 0.4505 - val_accuracy: 0.8936
Epoch 2/5
60000/60000 [==============================] - 3s 56us/sample - loss: 0.3748 - accuracy: 0.9089 - val_loss: 0.3705 - val_accuracy: 0.9128
Epoch 3/5
60000/60000 [==============================] - 3s 58us/sample - loss: 0.3050 - accuracy: 0.9260 - val_loss: 0.3273 - val_accuracy: 0.9220
Epoch 4/5
60000/60000 [==============================] - 3s 53us/sample - loss: 0.2741 - accuracy: 0.9340 - val_loss: 0.3798 - val_accuracy: 0.9065
Epoch 5/5
60000/60000 [==============================] - 3s 56us/sample - loss: 0.2543 - accuracy: 0.9387 - val_loss: 0.2959 - val_accuracy: 0.9369


In [124]:
#      CLONE MNIST MODEL EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)
#K.clear_session() 
tf.compat.v1.disable_eager_execution()

clone = Clone_Model.satify_model(model)

sat_cb = Clone_Model.sat_results()

history = clone.fit(train[0],
                    train[1],
                    batch_size=3,
                    epochs=1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[sat_cb])
                

<class 'tensorflow.python.keras.layers.core.Flatten'>
ListWrapper(['o_s', 'r_s', 's_s'])
ListWrapper(['o_s', 'r_s', 's_s'])
RESET Weights!
RESET Weights!
Clone_Model.satify_model() end.
Train on 3 samples, validate on 10000 samples
AGGREG UPDATESTATE() CALLED: 0.0
AGGREG UPDATESTATE() CALLED: 0.0
AGGREG UPDATESTATE() CALLED: Tensor("update_tensor:0", shape=(128, 128), dtype=float64)
AGGREG UPDATESTATE() CALLED: Tensor("update_tensor:0", shape=(128,), dtype=float64)
AGGREG UPDATESTATE() CALLED: Tensor("update_tensor:0", shape=(10, 10), dtype=float64)
AGGREG UPDATESTATE() CALLED: Tensor("update_tensor:0", shape=(10,), dtype=float64)

Logs from batch callback: {'batch': 0, 'size': 1, 'loss': 0.22063896, 'accuracy': 0.9450167}
1/3 [=========>....................] - ETA: 0s - loss: 0.2206 - accuracy: 0.9450AGGREG UPDATESTATE() CALLED: 0.0
AGGREG UPDATESTATE() CALLED: 0.0
AGGREG UPDATESTATE() CALLED: Tensor("update_tensor:0", shape=(128, 128), dtype=float64)
AGGREG UPDATESTATE() CALLED: Tens

In [125]:
print(history.history.keys())
print(history.__dict__)

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
{'validation_data': None, 'model': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fe02832fc90>, '_chief_worker_only': None, 'history': {'loss': [0.19146684805552164], 'accuracy': [0.9508833], 'val_loss': [0.24173198640346527], 'val_accuracy': [0.947]}, 'params': {'batch_size': 3, 'epochs': 1, 'steps': 3, 'samples': 3, 'verbose': 1, 'do_validation': True, 'metrics': ['loss', 'accuracy', 'o_sdense', 'o_sdense_1', 'val_loss', 'val_accuracy', 'val_o_sdense', 'val_o_sdense_1']}, 'epoch': [0]}


In [ ]:
""" Warum macht tf probleme und zeigt auf auskommentierten code?
        #Default to k=1 if first EVal explains delta of variance alone 
        #TODO change condition so graph mode works without tf.function
        #if not 
        #    #cond = tf.constant([True], dtype=tf.bool)
            #print("cond with default catch: {}".format(cond))   
        """    
        
    
    #k = max(len(tf.where(cond)), 1 ) #TODO use min(1,tf.where()) instead of checking above
        
        #print("tf.where(): {}".format(tf.where(cond)))
        #print("K: in ll_fun {}".format(k))
       